In [1]:
from qem.image_fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
%matplotlib qt

# file = '/home/zzhang/OneDrive/code/qem/data/STO/adf_average_STO.txt'

# image = np.loadtxt(file)
# file = '/home/zzhang/OneDrive/data/Nico/28.03.2024-STOonSTO21mrad/83-100/aligned_hdf5/1859 HAADF 33.7 nm.emd.hspy'
file = '1859 HAADF 33.7 nm.emd.hspy'
# file ='/home/zzhang/OneDrive/data/Nico/28.03.2024-STOonSTO21mrad/84-100/aligned_hdf5/1932 HAADF 33.7 nm.emd.hspy'

s = hs.load(file)
image = s.data
# image = np.load(file)
# dx= 0.1645429228960236	
dx = s.axes_manager[1].scale *10
# dx = 0.36254824166962796
model=ImageModelFitting(image, dx=dx,units='A')

# get the unique coordinates by row
# input_coordinates = np.unique(input_coordinates, axis=0)

INFO:root:Elements: ['Sr', 'Ti', 'O'], the order is used for the atom types. Please initiate the correct elements in your system.


In [ ]:
model.plot()

In [2]:
model.find_peaks(min_distance=10,threshold_abs=11600,threshold_rel=0.05)

INFO:root:Updated peak locations.


array([[ 613., 1575.],
       [ 907.,  820.],
       [ 396., 1306.],
       ...,
       [ 917.,  121.],
       [1421.,  206.],
       [1586.,  121.]])

In [ ]:
model.assign_region_label(1)

In [5]:
model.plot_region()

In [3]:
%matplotlib qt
cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
model.map_lattice(cif_file=cif_file,elements=['Sr','Ti'], reciprocal=True)

/home/zzhang/OneDrive/code/qem/qem/gui_classes.py:361: UserWarning: You passed a edgecolor/edgecolors ('black') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  plt.scatter(
INFO:root:Origin selected: [ 807. 1307.]
INFO:root:Vector a selected: [25.  2.]
INFO:root:Vector b selected: [  1. -22.]
INFO:root:In pixel: Origin: [ 807. 1307.], Vector a: [25.  2.], Vector b: [  1. -22.]
INFO:root:In space: Origin: [132.78613878 215.05760023] A, Vector a: [4.11357307 0.32908585] A, Vector b: [ 0.16454292 -3.6199443 ] A
INFO:root:Origin selected: [1024. 1108.]
INFO:root:Vector a selected: [84.  0.]
INFO:root:Vector b selected: [-1. 84.]
INFO:root:In pixel: Origin: [1024. 1108.], Vector a: [84.  0.], Vector b: [-1. 84.]
INFO:root:In space: Origin: [3.03872079 3.28799085] 1/A, Vector a: [0.24927006 0.        ] 1/A, Vector b: [-0.0029675   0.24927006] 1/A
INFO:root:FFT real a: [24.38095238  0.        ] pixel, Re

AtomicColumns(lattice=Atoms(symbols='Sr6364Ti6399', pbc=False, cell=[336.98390609105627, 336.98390609105627, 3.94513], spacegroup_kinds=...), lattice_ref=Atoms(symbols='Sr6364Ti6399', pbc=False, cell=[336.98390609105627, 336.98390609105627, 3.94513], spacegroup_kinds=...), elements=['Sr', 'Ti'], tol=0, pixel_size=0.16454292289602357, reference={'origin': array([ 807., 1307.]), 'vector_a': array([23.97629707,  0.        ]), 'vector_b': array([ 0.        , 23.97629707])})

In [8]:
model.region_crysal_analyzer[0].atomic_columns

AtomicColumns(lattice=Atoms(symbols='Sr6370Ti6393', pbc=False, cell=[336.98390609105627, 336.98390609105627, 3.94513], spacegroup_kinds=...), lattice_ref=Atoms(symbols='Sr6370Ti6393', pbc=False, cell=[336.98390609105627, 336.98390609105627, 3.94513], spacegroup_kinds=...), elements=['Sr', 'Ti'], tol=0, pixel_size=0.16454292289602357, reference={'origin': array([ 710., 1405.]), 'vector_a': array([23.97629707,  0.        ]), 'vector_b': array([ 0.        , 23.97629707])})

In [16]:
from matscipy.neighbours import neighbour_list
i,j = neighbour_list('ij', model.region_crysal_analyzer[0].atomic_columns.lattice, 5.0)
coord = np.bincount(i)

In [4]:
model.region_crysal_analyzer[0].atomic_columns.lattice.get_global_number_of_atoms()

12763

In [26]:
i.shape

(100460,)

In [25]:
coord.shape

(12763,)

In [21]:
j.shape

(100460,)

In [5]:
model.region_crysal_analyzer[0].plot_displacement(cut_off=5.0)

/home/zzhang/miniconda3/envs/jax/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/zzhang/miniconda3/envs/jax/lib/python3.11/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


KeyboardInterrupt: 

In [ ]:
model.find_peaks(min_distance=10,threshold_abs=11600,threshold_rel=0.05,region_index=1)
model.map_lattice(cif_file=cif_file,elements=['Sr','Ti'], reciprocal=True,region_index=1)

In [ ]:
from ase.visualize import view
view(model.region_atomic_column[0].lattice)

In [ ]:
model.add_or_remove_peaks(min_distance=10)

In [9]:
model.plot_coordinates(10)

In [ ]:
model.fit_background = True
params = model.init_params()
model.fit_random_batch(params, batch_size=1000, maxiter=50, tol=1e-3, plot=False, step_size=0.01,verbose=False,num_epoch=1)

In [12]:
model.plot_fitting()

In [ ]:
plt.figure()
plt.hist(model.volume[model.atom_types==0], bins=100, alpha=0.5,label='Sr')
plt.hist(model.volume[model.atom_types==1], bins=100, alpha=0.5,label='Ti')
plt.legend()

In [7]:
model.plot_scs(per_element=True)

In [15]:
model.plot_scs_histogram()

In [ ]:
import numpy as np

# Sample array
arr = np.array([2, 3, 2, 1, 3, 1, 4])

# Get unique values, indices of first occurrence, and inverse mapping
unique_values, first_indices, inverse_indices = np.unique(arr, return_index=True, return_inverse=True)

# Create a dictionary to store the indices of all occurrences for each unique value
occurrences = {value: np.where(inverse_indices == i)[0] for i, value in enumerate(unique_values)}

print("Unique values:", unique_values)
print("Indices of first occurrence:", first_indices)
print("Indices of all occurrences for each unique value:", occurrences)


In [ ]:
import numpy as np
from scipy.cluster.hierarchy import fcluster, linkage

# Sample array with values that need tolerance larger than 1
arr = np.array([1, 2, 2.5, 3.2, 10, 10.5, 15])

# Set the desired tolerance (e.g., values within 3 units are considered the same)
tolerance = 3

# Perform hierarchical clustering
Z = linkage(arr[:, None], method='single', metric='euclidean')

# Create clusters based on the tolerance
clusters = fcluster(Z, t=tolerance, criterion='distance')

# Find unique cluster labels and indices of first occurrence
_, unique_indices = np.unique(clusters, return_index=True)

# Get unique values based on clusters
unique_values = arr[unique_indices]

print("Unique values with tolerance:", unique_values)
print("Indices of first occurrence:", unique_indices)
